In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [11]:
from langchain_groq import ChatGroq
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [22]:
embedding=OllamaEmbeddings(model="mxbai-embed-large")

C:\Users\user\AppData\Local\Temp\ipykernel_10616\1174451878.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model="mxbai-embed-large")


In [5]:
llm=ChatGroq(model="gemma2-9b-it")

In [6]:
llm.invoke("hello")

AIMessage(content='Hello! 👋\n\nHow can I help you today? 😄\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 10, 'total_tokens': 25, 'completion_time': 0.027272727, 'prompt_time': 3.1e-07, 'queue_time': 0.021000005, 'total_time': 0.027273037}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1e56ceb0-42d3-464a-917b-63e3b0eac17a-0', usage_metadata={'input_tokens': 10, 'output_tokens': 15, 'total_tokens': 25})

In [13]:
loader=TextLoader('sample.txt')
docs=loader.load()
docs

[Document(metadata={'source': 'sample.txt'}, page_content='The definition of Deep learning is that it is the branch of machine learning that is based on artificial neural network architecture. An artificial neural network or ANN uses layers of interconnected nodes called neurons that work together to process and learn from the input data.\n\nIn a fully connected Deep neural network, there is an input layer and one or more hidden layers connected one after the other. Each neuron receives input from the previous layer neurons or the input layer. The output of one neuron becomes the input to other neurons in the next layer of the network, and this process continues until the final layer produces the output of the network. The layers of the neural network transform the input data through a series of nonlinear transformations, allowing the network to learn complex representations of the input data.\n\nDeep Learning - Geeksforgeeks\nScope of Deep Learning\n\nToday Deep learning AI has become

In [17]:
splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
document=splitter.split_documents(docs)

In [21]:
document[0]

Document(metadata={'source': 'sample.txt'}, page_content='The definition of Deep learning is that it is the branch of machine learning that is based on artificial neural network architecture. An artificial neural network or ANN uses layers of interconnected nodes called neurons that work together to process and learn from the input data.')

In [23]:
#vectorstor
vectorstore=FAISS.from_documents(documents=document,embedding=embedding)

In [24]:
#similiarity search

question="what is deep learning"
result=vectorstore.similarity_search(question)
print(result)

[Document(id='1e933c12-5d4e-4917-843e-15a42b9301bc', metadata={'source': 'sample.txt'}, page_content='The definition of Deep learning is that it is the branch of machine learning that is based on artificial neural network architecture. An artificial neural network or ANN uses layers of interconnected nodes called neurons that work together to process and learn from the input data.'), Document(id='623c0d49-9937-4b51-b46b-24250a01807e', metadata={'source': 'sample.txt'}, page_content='Deep Learning - Geeksforgeeks\nScope of Deep Learning\n\nToday Deep learning AI has become one of the most popular and visible areas of machine learning, due to its success in a variety of applications, such as computer vision, natural language processing, and Reinforcement learning.\n\nDeep learning AI can be used for supervised, unsupervised as well as reinforcement machine learning. it uses a variety of ways to process these.'), Document(id='2d1c7521-e13e-4b3a-8b90-09312f489ce2', metadata={'source': 'sam

In [26]:
prompt=ChatPromptTemplate.from_template(
    """
        Answer the following question based on the given context:
        <context>
        {context}
        <context>      
    """
)                                        

In [28]:
#converting vector database to retriever it act as a interface 
retriever=vectorstore.as_retriever() 

In [30]:
retriever.get_relevant_documents("what is deep learning")

[Document(id='1e933c12-5d4e-4917-843e-15a42b9301bc', metadata={'source': 'sample.txt'}, page_content='The definition of Deep learning is that it is the branch of machine learning that is based on artificial neural network architecture. An artificial neural network or ANN uses layers of interconnected nodes called neurons that work together to process and learn from the input data.'),
 Document(id='623c0d49-9937-4b51-b46b-24250a01807e', metadata={'source': 'sample.txt'}, page_content='Deep Learning - Geeksforgeeks\nScope of Deep Learning\n\nToday Deep learning AI has become one of the most popular and visible areas of machine learning, due to its success in a variety of applications, such as computer vision, natural language processing, and Reinforcement learning.\n\nDeep learning AI can be used for supervised, unsupervised as well as reinforcement machine learning. it uses a variety of ways to process these.'),
 Document(id='2d1c7521-e13e-4b3a-8b90-09312f489ce2', metadata={'source': 's

In [32]:
#it creates a chain for passing a list of documents to a model because whenever we are 
#retrieving the content the content will be multiple documents 
combine_docs=create_stuff_documents_chain(llm=llm,prompt=prompt)
print(combine_docs)

bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based on the given context:\n        <context>\n        {context}\n        <context>      \n    '), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022149E8C140>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022149E8CC50>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser() kwargs={} config={'run_name': 'stuff_documents_chain'} config_factories=[]


In [34]:
#it used to comine both stuff_document_chain and retrieval_chain
chain=create_retrieval_chain(retriever,combine_docs)
print(chain)

bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002214BDADA00>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based on the given context:\n        <context>\n        {context}\n        <context>      \n    '), additional_kwargs={})])
 

In [38]:
result=chain.invoke({"input":"what is deep learning"})

In [39]:
result['answer']

'According to the provided text, Deep Learning can be used for **supervised, unsupervised, and reinforcement machine learning**. \n\n\nLet me know if you have any other questions! \n'